___
# Jovian analysis report
___

**This report should be generated automatically. However, if it somehow doesn't then please follow the steps below to view your results:**  
<br> 
**- You can generate the report by pressing `Cell` in the toolbar and then selecting `Run All`. This can take a couple of minutes (depending on the size of your dataset). As long as the circle in the top right corner is not-empty (i.e. completely filled), the data is still being loaded.**  
<br>
**- If you want to show or hide the programming code, press the button with the eye in it on the toolbar above.**  
<br>
**- N.B. The sum total of reads in this report will not add up to the sum total number of reads that were supplied as input. This is because, 1) human reads are removed, 2) PCR-duplicate reads are removed.**  
<br>
You can view the graph displaying all Jovian's disparate processes by clicking [here](https://github.com/DennisSchmitz/Jovian/blob/assets/images/rulegraph_Jovian.png?raw=true).

In [ ]:
%%html
<style>
div.output_subarea {
    padding-top: 0 !important;
    padding-bottom: 0 !important;
}

.standardheader {
    font-size: 30px !important;
    padding: 0px;
}

.standardanchor {
    font-size: 14px !important;
    margin-left: 5px;
}
.standardtext {
    font-family: sans-serif;
    font-size: 14px !important;
    margin-left: 5px;
}

In [ ]:
######################################
# Required packages for this script  #
######################################
import pandas as pd
import qgrid
import glob
import os
from IPython.display import display as dp
from IPython.display import Markdown as md
from IPython.display import IFrame as fr
from IPython.display import HTML as ht
import plotly as pl
import plotly.graph_objects as go
import plotly.express as px


### Do Bash things
!bash bin/scripts/set_symlink.sh
!bash bin/includes/Start_nginx start

### Standard Qgrid options
grid_options = {
    'fullWidthRows': True,
    'syncColumnCellResize': True,
    'forceFitColumns': False,
    'defaultColumnWidth': 100,
    'rowHeight': 23,
    'enableColumnReorder': True,
    'enableTextSelectionOnCells': True,
    'editable': True,
    'autoEdit': False,
    'explicitInitialization': True,
    'maxVisibleRows': 20,
    'minVisibleRows': 8,
    'sortable': True,
    'filterable': True,
    'highlightSelectedCell': True,
    'highlightSelectedRow': True,
    'show_toolbar': False
}

# Qgrid options for the BoC tables
boc_grid_options = {
    'fullWidthRows': True,
    'syncColumnCellResize': True,
    'forceFitColumns': True,
    'defaultColumnWidth': 100,
    'rowHeight': 23,
    'enableColumnReorder': True,
    'enableTextSelectionOnCells': True,
    'editable': True,
    'autoEdit': False,
    'explicitInitialization': True,
    'maxVisibleRows': 20,
    'minVisibleRows': 8,
    'sortable': True,
    'filterable': True,
    'highlightSelectedCell': True,
    'highlightSelectedRow': True,
    'show_toolbar': False
}


######################################
# TOC CONTENTS                       #
######################################

toc_header = '''
***
# Table of Contents:
'''
toc_footer = "***"

goback = "[Go back to the Table of Contents](#Jovian-analysis-report)  "

qc_toc_intro = "### Quality control metrics:"
multiqc_link = "* [Quality control metrics report (MultiQC)](#quality-control-metrics-report)"
composition_link = "* [Read based composition of analyzed samples](#read-based-composition-of-analyzed-samples-)"

meta_toc_intro = "### Metagenomics"
krona_link = "* [Interactive metagenomics overview](#interactive-metagenomics-overview-krona-)"

hm_toc_intro = "#### &nbsp;&nbsp; Heatmaps"
hm_sup_link = "* * [Superkingdom heatmap](#superkingdom-heatmap)"
hm_vir_link = "* * [Virus heatmap](#virus-heatmaps)"
hm_pha_link = "* * [Phage heatmap](#phage-heatmaps)"
hm_bac_link = "* * [Bacteria heatmap](#bacteria-heatmaps)"

taxclas_toc_intro = "### Taxonomic classification"
taxclas_toc_link = "* [Classified scaffolds](#classified-scaffolds-)"
taxunclas_toc_link = "* [Unclassified scaffolds](#unclassified-scaffolds)"

viro_toc_intro = "### Viral scaffold information"
virhost_toc_link = "* [Predicted virus hosts](#predicted-virus-hosts-)"
vir_typing_toc_intro = "#### &nbsp;&nbsp; Viral typing results"
vir_typing_nov_link = "* * [Norovirus typing tool results](#norovirus-typing-tool-output-)"
vir_typing_rva_link = "* * [Rotavirus typing tool results](#rotavirus-a-typing-tool-output-)"
vir_typing_env_link = "* * [Enterovirus typing tool results](#enterovirus-typing-tool-output-)"
vir_typing_hav_link = "* * [Hepatitis A typing tool results](#hepatitis-a-typing-tool-output-)"
vir_typing_hev_link = "* * [Hepatitis E typing tool results](#hepatitis-e-typing-tool-output-)"
vir_typing_hpv_link = "* * [Human Papillomavirus typing tool results](#human-papillomavirus-typing-tool-output-)"
vir_typing_fla_link = "* * [Flavirus typing tool results](#flavivirus-typing-tool-output-)"

scf_viewer_link = "* [Interactive Scaffold viewer](#scaffold-viewer-)"

snp_table_link = "* [Minority variant table (SNPs)](#minority-variant-table-)"


######################################
# ACTUAL CONTENT                     #
######################################

mqc_cell = '''
___
## Quality control metrics report  
[Open MultiQC graph in separate tab by clicking here](results/multiqc.html)  
'''

composition_cell = '''
___
### Read-based composition of analyzed samples:  
[Open the barchart in separate tab by clicking here](results/Sample_composition_graph.html)  
<br>
**Low-quality** reads are those that did not meet the stringency settings as specified in the config file.  
**Unclassified** reads are those that could not be assigned to a taxa.  
**Remaining** reads are those that could not be assembled into contigs longer than the user specified minimum contig length. 
'''

krona_cell = '''
___
## Metagenomics:
___
<br>
<br>

### Interactive metagenomics overview (Krona):
[Open Krona graph in separate tab by clicking here](results/krona.html)  
'''

heatmap_cell_header = '''
### Heatmaps:
'''

heatmap_cell_sup = '''
#### Superkingdom heatmap  
Open superkingdoms heatmap in a separate tab by clicking [here](results/heatmaps/Superkingdoms_heatmap.html).  
'''

heatmap_cell_vir = '''
#### Virus heatmaps
Open virus heatmap in a separate tab by clicking [here](results/heatmaps/Virus_heatmap.html).  
<br>
**Please note, many viruses have no "`order`" taxonomic rank so always check the "`family`" taxonomic rank.**
'''

heatmap_cell_pha = '''
#### Phage heatmaps
Open phage heatmap in a separate tab by clicking [here](results/heatmaps/Phage_heatmap.html).  
<br>
**Please note, many viruses have no "`order`" taxonomic rank so always check the "`family`" taxonomic rank.**
'''

heatmap_cell_bac = '''
#### Bacteria heatmaps
Open bacteria heatmap in a separate tab by clicking [here](results/heatmaps/Bacteria_heatmap.html).  
'''

clas_scf_cell = '''
___
### Classified scaffolds:
'''

unclas_scf_cell = '''
___
### Unclassified scaffolds ("Dark Matter"):
'''

virhosts_cell = '''
___
## Predicted virus hosts:
___
'''

virtyping_cell = '''
___
## Virus typing results:
___
<br>
The virus typing outputs are currently not automatically generated via Jovian due to overloading and crashing the web-service. A long-term solution is being worked on, as a work-around we've included scripts to generate these data separately. However, we kindly ask you to <u>**use this sparingly**</u> as to not overload and break the web-service. Instructions on how to generate these data can be found on the [website](https://github.com/DennisSchmitz/Jovian), [here](https://github.com/DennisSchmitz/Jovian/issues/29) and [here](https://github.com/DennisSchmitz/Jovian/issues/51).
'''

virtyping_nov_cell = '''
### Norovirus typing tool output:  
[Link to the norovirus typing tool](https://www.rivm.nl/mpf/typingtool/norovirus/)  
'''

virtyping_rva_cell = '''
### Rotavirus A typing tool output:  
[Link to the Rotavirus A typing tool](https://www.rivm.nl/mpf/typingtool/rotavirusa/)  
'''

virtyping_env_cell = '''
### Enterovirus typing tool output:  
[Link to the enterovirus typing tool](https://www.rivm.nl/mpf/typingtool/enterovirus/)  
'''

virtyping_hav_cell = '''
### Hepatitis A typing tool output:  
[Link to the hepatatis A typing tool](https://www.rivm.nl/mpf/typingtool/hav/)  
'''

virtyping_hev_cell = '''
### Hepatitis E typing tool output:  
[Link to the hepatatis E typing tool](https://www.rivm.nl/mpf/typingtool/hev/)  
'''

virtyping_hpv_cell = '''
### Human Papillomavirus typing tool output:  
[Link to the HPV typing tool](https://www.rivm.nl/mpf/typingtool/papillomavirus/)  
'''

virtyping_fla_cell = '''
### Flavivirus typing tool output:  
[Link to the flavivirus typing tool](https://www.rivm.nl/mpf/typingtool/flavivirus/)  
'''

boc_header_cell = '''
___
## Breadth of Coverage reports:
___
'''


scf_viewer_cell  = '''
___
## Scaffold viewer:
**Containing: SNPs and minority variants (quasispecies), predicted ORFs, depth of coverage graph, GC contents graph**
___
N.B. Depending on the depth of coverage of the selected contig it can be <b>(very) slow, or downright crash your browser</b>. This is a <b>client-sided</b> problem, meaning, your computer isn't powerful enough.  
Open the scaffold viewer in a separate tab by clicking [here](results/IGVjs_index.html).  
'''

snp_table_cell = '''
___
## Minority variant table:
___
'''

snp_table_empty_cell = '''
Either no SNP's were classified, maybe because you've set the minimum allele-frequency too high? Or something went wrong, please doublecheck the logfiles below:
    **logs/SNP_calling_[sample_name].log**
    **logs/Concat_filtered_SNPs.log**
'''

logging_header_cell = '''
___
# Logging and audit-trail: 
___
'''

snakemake_report_cell = '''
### Snakemake summary statistics
[Open Snakemake summary statistics in separate tab by clicking here](snakemake_report.html#stats)
'''

logfile_index = '''
<script>
function goBack() {
    window.history.back()
}
</script>

<button onclick="goBack()">Click this button to go back</button>

<div style="text-align: center">
    <iframe src="results/logfiles_index.html" width=100% height=980></iframe>
</div>
'''

acknowledgements_header_cell = '''
___
# Acknowledgements:
___
'''

######################################
# WRITE-FUNCTIONS                    #
######################################

def toc():
    dp(md(toc_header))
    if os.path.exists("results/multiqc.html"):
        dp(md(qc_toc_intro))
        dp(md(multiqc_link))
    if os.path.exists("results/Sample_composition_graph.html"):
        dp(md(composition_link))
    if os.path.exists("results/krona.html"):
        dp(md(meta_toc_intro))
        dp(md(krona_link))
    if os.path.exists("results/heatmaps"):
        dp(md(hm_toc_intro))
        if os.path.exists("results/heatmaps/Superkingdoms_heatmap.html"):
            dp(md(hm_sup_link))
        if os.path.exists("results/heatmaps/Virus_heatmap.html"):
            dp(md(hm_vir_link))
        if os.path.exists("results/heatmaps/Phage_heatmap.html"):
            dp(md(hm_pha_link))
        if os.path.exists("results/heatmaps/Bacteria_heatmap.html"):
            dp(md(hm_bac_link))
    if os.path.exists("results/all_taxClassified.tsv"):
        dp(md(taxclas_toc_intro))
        dp(md(taxclas_toc_link))
        dp(md(taxunclas_toc_link))
    if os.path.exists("results/all_virusHost.tsv") or os.path.exists("results/typingtools"):
        dp(md(viro_toc_intro))
        if os.path.exists("results/all_virusHost.tsv"):
            dp(md(virhost_toc_link))
        if os.path.exists("results/typingtools"):
            dp(md(vir_typing_toc_intro))
            if os.path.exists("results/typingtools/all_nov-TT.csv") and os.path.getsize("results/typingtools/all_nov-TT.csv") > 0:
                dp(md(vir_typing_nov_link))
            if os.path.exists("results/typingtools/all_rva-TT.csv") and os.path.getsize("results/typingtools/all_rva-TT.csv") > 0:
                dp(md(vir_typing_rva_link))
            if os.path.exists("results/typingtools/all_ev-TT.csv") and os.path.getsize("results/typingtools/all_ev-TT.csv") > 0:
                dp(md(vir_typing_env_link))
            if os.path.exists("results/typingtools/all_hav-TT.csv") and os.path.getsize("results/typingtools/all_hav-TT.csv") > 0:
                dp(md(vir_typing_hav_link))
            if os.path.exists("results/typingtools/all_hev-TT.csv") and os.path.getsize("results/typingtools/all_hev-TT.csv") > 0:
                dp(md(vir_typing_hev_link))
            if os.path.exists("results/typingtools/all_pv-TT.csv") and os.path.getsize("results/typingtools/all_pv-TT.csv") > 0:
                dp(md(vir_typing_hpv_link))
            if os.path.exists("results/typingtools/all_flavi-TT.csv") and os.path.getsize("results/typingtools/all_flavi-TT.csv") > 0:
                dp(md(vir_typing_fla_link))
    if os.path.exists("results/igv.html"):
        dp(md(scf_viewer_link))
    if os.path.exists("results/all_filtered_SNPs.tsv") and os.path.getsize("results/all_filtered_SNPs.tsv") > 0:
        dp(md(snp_table_link))
    dp(md(toc_footer))


    
    

def multiqc():
    if os.path.exists("results/multiqc.html"):
        dp(md(mqc_cell))
        dp(fr('results/multiqc.html', '100%', '980px'))
        dp(md(goback))
        
def composition():
    if os.path.exists("results/Sample_composition_graph.html"):
        dp(md(composition_cell))
        dp(fr('results/Sample_composition_graph.html', '100%', '980px'))
        dp(md(goback))

def krona():
    if os.path.exists("results/krona.html"):
        dp(md(krona_cell))
        dp(fr('results/krona.html', '100%', '980px'))
        dp(md(goback))

def heatmaps():
    if os.path.exists("results/heatmaps"):
        dp(md(heatmap_cell_header))
        if os.path.exists("results/heatmaps/Superkingdoms_heatmap.html"):
            dp(md(heatmap_cell_sup))
            dp(fr('results/heatmaps/Superkingdoms_heatmap.html', '100%', '700px'))
            dp(md(goback))
        if os.path.exists("results/heatmaps/Virus_heatmap.html"):
            dp(md(heatmap_cell_vir))
            dp(fr('results/heatmaps/Virus_heatmap.html', '100%', '700px'))
            dp(md(goback))
        if os.path.exists("results/heatmaps/Phage_heatmap.html"):
            dp(md(heatmap_cell_pha))
            dp(fr('results/heatmaps/Phage_heatmap.html', '100%', '700px'))
            dp(md(goback))
        if os.path.exists("results/heatmaps/Bacteria_heatmap.html"):
            dp(md(heatmap_cell_bac))
            dp(fr('results/heatmaps/Bacteria_heatmap.html', '100%', '700px'))
            dp(md(goback))

def scaffoldviewer():
    if os.path.exists("results/igv.html"):
        dp(md(scf_viewer_cell))
        dp(fr('results/igv.html', '100%', '980px'))
        dp(md(goback))


def audit():
    dp(md(logging_header_cell))
    dp(md("### Sample sheet"))
    sheet = open("sample_sheet.yaml", "r")
    print(sheet.read())
    dp(md(snakemake_report_cell))
    if os.path.exists("results/snakemake_report.html"):
        dp(fr('results/snakemake_report.html', '100%', '980px'))
    else:
        print("""
        We couldn't find a valid snakemake report.
        This indicates that something went wrong during the used workflow. Please run your analysis again.
        """)
    dp(md("### All log-files:"))
    dp(ht(logfile_index))
    dp(md("### Full software list in \"Jovian_master\" environment:"))
    conda_log = open("results/log_conda.txt", "r")
    print(conda_log.read())
    dp(md("### Database versions:"))
    db_log = open("results/log_db.txt", "r")
    print(db_log.read())
    dp(md("### Unique methodological \"fingerprint\":"))
    id_log = open("results/log_git.txt", "r")
    print(id_log.read())
    dp(md("### Snakemake config files:"))
    conf_log = open("results/log_config.txt", "r")
    print(conf_log.read())
    dp(md(goback))


def acks():
    dp(md(acknowledgements_header_cell))
    with open("files/acknowledgements.md", "r") as ack_base:
        ack_list = ack_base.read()
    dp(md(ack_list))
    with open("files/authors.md", "r") as ath_base:
        ath_list = ath_base.read()
    dp(md(ath_list))


### interactive tables and/or grids            
class Classified_grid:
    def dataframe(clas):
        if os.path.exists("results/all_taxClassified.tsv"):
            clas.df = pd.read_csv('results/all_taxClassified.tsv', sep = "\t")
            clas.grid = qgrid.show_grid(clas.df, grid_options=grid_options)
            return
    def draw(clas):
        clas.dataframe()
        if os.path.exists("results/all_taxClassified.tsv"):
            dp(md(clas_scf_cell))
            dp(clas.grid)
            dp(md(goback))

class Unclassified_grid:
    def dataframe(unclas):
        if os.path.exists("results/all_taxUnclassified.tsv"):
            unclas.df = pd.read_csv('results/all_taxUnclassified.tsv', sep = "\t")
            unclas.grid = qgrid.show_grid(unclas.df, grid_options=grid_options)
            return
    def draw(unclas):
        unclas.dataframe()
        if os.path.exists("results/all_taxClassified.tsv"):
            dp(md(unclas_scf_cell))
            dp(unclas.grid)
            dp(md(goback))

class Predict_virhosts:
    def dataframe(predicts):
        if os.path.exists("results/all_virusHost.tsv"):
            predicts.df = pd.read_csv('results/all_virusHost.tsv', sep = "\t")
            predicts.grid = qgrid.show_grid(predicts.df, grid_options=grid_options)
            return
    def draw(predicts):
        predicts.dataframe()
        if os.path.exists("results/all_virusHost.tsv"):
            dp(md(virhosts_cell))
            dp(predicts.grid)
            dp(md(goback))

def virtypingheader():
    if os.path.exists("results/typingtools"):
        dp(md(virtyping_cell))


class nov_typing:
    def dataframe(nov):
        if os.path.exists("results/typingtools/all_nov-TT.csv") and os.path.getsize("results/typingtools/all_nov-TT.csv") > 0:
            nov.df = pd.read_csv('results/typingtools/all_nov-TT.csv')
            nov.grid = qgrid.show_grid(nov.df, grid_options=grid_options)
            return
    def draw(nov):
        nov.dataframe()
        if os.path.exists("results/typingtools/all_nov-TT.csv") and os.path.getsize("results/typingtools/all_nov-TT.csv") > 0:
            dp(md(virtyping_nov_cell))
            dp(nov.grid)
            dp(md(goback))

class rva_typing:
    def dataframe(rva):
        if os.path.exists("results/typingtools/all_rva-TT.csv") and os.path.getsize("results/typingtools/all_rva-TT.csv") > 0:
            rva.df = pd.read_csv('results/typingtools/all_rva-TT.csv')
            rva.grid = qgrid.show_grid(rva.df, grid_options=grid_options)
            return
    def draw(rva):
        rva.dataframe()
        if os.path.exists("results/typingtools/all_rva-TT.csv") and os.path.getsize("results/typingtools/all_rva-TT.csv") > 0:
            dp(md(virtyping_rva_cell))
            dp(rva.grid)
            dp(md(goback))

class env_typing:
    def dataframe(env):
        if os.path.exists("results/typingtools/all_ev-TT.csv") and os.path.getsize("results/typingtools/all_ev-TT.csv") > 0:
            env.df = pd.read_csv('results/typingtools/all_ev-TT.csv')
            env.grid = qgrid.show_grid(env.df, grid_options=grid_options)
            return
    def draw(env):
        env.dataframe()
        if os.path.exists("results/typingtools/all_ev-TT.csv") and os.path.getsize("results/typingtools/all_ev-TT.csv") > 0:
            dp(md(virtyping_env_cell))
            dp(env.grid)
            dp(md(goback))

class hav_typing:
    def dataframe(hav):
        if os.path.exists("results/typingtools/all_hav-TT.csv") and os.path.getsize("results/typingtools/all_hav-TT.csv") > 0:
            hav.df = pd.read_csv('results/typingtools/all_hav-TT.csv')
            hav.grid = qgrid.show_grid(hav.df, grid_options=grid_options)
            return
    def draw(hav):
        hav.dataframe()
        if os.path.exists("results/typingtools/all_hav-TT.csv") and os.path.getsize("results/typingtools/all_hav-TT.csv") > 0:
            dp(md(virtyping_hav_cell))
            dp(hav.grid)
            dp(md(goback))

class hev_typing:
    def dataframe(hev):
        if os.path.exists("results/typingtools/all_hev-TT.csv") and os.path.getsize("results/typingtools/all_hev-TT.csv") > 0:
            hev.df = pd.read_csv('results/typingtools/all_hev-TT.csv')
            hev.grid = qgrid.show_grid(hev.df, grid_options=grid_options)
            return
    def draw(hev):
        hev.dataframe()
        if os.path.exists("results/typingtools/all_hev-TT.csv") and os.path.getsize("results/typingtools/all_hev-TT.csv") > 0:
            dp(md(virtyping_hev_cell))
            dp(hev.grid)
            dp(md(goback))

class hpv_typing:
    def dataframe(hpv):
        if os.path.exists("results/typingtools/all_pv-TT.csv") and os.path.getsize("results/typingtools/all_pv-TT.csv") > 0:
            hpv.df = pd.read_csv('results/typingtools/all_pv-TT.csv')
            hpv.grid = qgrid.show_grid(hpv.df, grid_options=grid_options)
            return
    def draw(hpv):
        hpv.dataframe()
        if os.path.exists("results/typingtools/all_pv-TT.csv") and os.path.getsize("results/typingtools/all_pv-TT.csv") > 0:
            dp(md(virtyping_hpv_cell))
            dp(hpv.grid)
            dp(md(goback))

class fla_typing:
    def dataframe(fla):
        if os.path.exists("results/typingtools/all_flavi-TT.csv") and os.path.getsize("results/typingtools/all_flavi-TT.csv") > 0:
            fla.df = pd.read_csv('results/typingtools/all_flavi-TT.csv')
            fla.grid = qgrid.show_grid(fla.df, grid_options=grid_options)
            return
    def draw(fla):
        fla.dataframe()
        if os.path.exists("results/typingtools/all_flavi-TT.csv") and os.path.getsize("results/typingtools/all_flavi-TT.csv") > 0:
            dp(md(virtyping_fla_cell))
            dp(fla.grid)
            dp(md(goback))

class snp_variants:
    def dataframe(snps):
        if os.path.exists("results/all_filtered_SNPs.tsv") and os.path.getsize("results/all_filtered_SNPs.tsv") > 0:
            snps.df = pd.read_csv('results/all_filtered_SNPs.tsv', sep="\t")
            snps.grid = qgrid.show_grid(snps.df, grid_options=grid_options)
            return
    def draw(snps):
        snps.dataframe()
        if os.path.exists("results/all_filtered_SNPs.tsv") and os.path.getsize("results/all_filtered_SNPs.tsv") > 0:
            dp(md(snp_table_cell))
            dp(snps.grid)
            dp(md(goback))
        else:
            dp(md(snp_table_cell))
            dp(md(snp_table_empty_cell))

def bocheaders():
    if os.path.exists("results/BoC_int.tsv") or os.path.exists("results/BoC_pct.tsv"):
        dp(md(boc_header_cell))
            
class breadth_of_cov_ints:
    def dataframe(ints):
        if os.path.exists("results/BoC_int.tsv") and os.path.getsize("results/BoC_int.tsv") > 0:
            ints.df = pd.read_csv('results/BoC_int.tsv', sep="\t")
            ints.df.set_index('Sample_name', inplace=True)
            ints.grid = qgrid.show_grid(ints.df, grid_options=boc_grid_options)
            return
    def draw(ints):
        ints.dataframe()
        if os.path.exists("results/BoC_int.tsv") and os.path.getsize("results/BoC_int.tsv") > 0:
            dp(md("### Breadth of Coverage overview in nucleotide-numbers"))
            dp(ints.grid)
            dp(md(goback))

class breadth_of_cov_pcts:
    def dataframe(pcts):
        if os.path.exists("results/BoC_pct.tsv") and os.path.getsize("results/BoC_pct.tsv") > 0:
            pcts.df = pd.read_csv('results/BoC_pct.tsv',  sep="\t")
            pcts.df.set_index('Sample_name', inplace=True)
            pcts.grid = qgrid.show_grid(pcts.df, grid_options=boc_grid_options)
            return
    def draw(pcts):
        pcts.dataframe()
        if os.path.exists("results/BoC_pct.tsv") and os.path.getsize("results/BoC_pct.tsv") > 0:
            dp(md("### Breath of Coverage overview in percentages"))
            dp(pcts.grid)
            dp(md(goback))

classified_scaffolds = Classified_grid()
unclassified_scaffolds = Unclassified_grid()
predicted_virushosts = Predict_virhosts()
virus_typing_nov = nov_typing()
virus_typing_rva = rva_typing()
virus_typing_env = env_typing()
virus_typing_hav = hav_typing()
virus_typing_hev = hev_typing()
virus_typing_hpv = hpv_typing()
virus_typing_fla = fla_typing()
minority_variant_table = snp_variants()
boc_table_ints = breadth_of_cov_ints()
boc_table_pcts = breadth_of_cov_pcts()

### actually draw the contents

#toc()
multiqc()
composition()
krona()
heatmaps()

classified_scaffolds.draw()
unclassified_scaffolds.draw()

predicted_virushosts.draw()

virtypingheader()
virus_typing_nov.draw()
virus_typing_rva.draw()
virus_typing_env.draw()
virus_typing_hav.draw()
virus_typing_hev.draw()
virus_typing_hpv.draw()
virus_typing_fla.draw()

bocheaders()
boc_table_ints.draw()
boc_table_pcts.draw()


scaffoldviewer()

minority_variant_table.draw()

audit()
acks()

___
Jovian is available on [GitHub](https://github.com/DennisSchmitz/Jovian) under a [AGPL license](https://www.gnu.org/licenses/agpl-3.0). The virus-typing tools are public services hosted by the [RIVM](https://www.rivm.nl/en) and developed independently of Jovian.
___

In [ ]:
dp(md(goback))

In [ ]:
%%javascript
$('<div id="toc"></div>').css({position: 'fixed', top: '120px', left: 0}).appendTo(document.body);
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js');